## Week 10 and 11 Assignment - DATASCI200 Introduction to Data Science Programming, UC Berkeley MIDS

Write code in this Jupyter Notebook to solve the following problems. Please upload this **Notebook** with your solutions to your GitHub repository and provide a link in the last question in gradescope. 

Assignment due date: 11:59PM PT the night before the Week 12 Live Session. Do **NOT** push/upload the data file. 

## Objectives

- Explore and glean insights from a real dataset using pandas
- Practice using pandas for exploratory analysis, information gathering, and discovery
- Practice cleaning data and answering questions

## General Guidelines:

- This is a **real** dataset and so it may contain errors and other pecularities to work through
- This dataset is ~218mb, which will take some time to load (and probably won't load in Google Sheets or Excel)
- If you make assumptions, annotate them in your responses
- While there is one code/markdown cell positioned after each question as a placeholder, some of your code/responses may require multiple cells
- Double-click the markdown cells that say for example **1a answer here:** to enter your written answers. If you need more cells for your written answers, make them markdown cells (rather than code cells)
- This homework assignment is not autograded because of the variety of responses one could give. 
  - Please upload this notebook to the autograder page and the TAs will manually grade it. 
  - Ensure that each cell is run and outputs your answer for ease of grading! 
  - Highly suggest to do a `restart & run all` before uploading your code to ensure everything runs and outputs correctly.
  - Answers without code (or code that runs) will be given 0 points.
- **This is meant to simulate real world data so you will have to do some external research to determine what some of the answers are!** 

## Dataset

You are to analyze campaign contributions to the 2016 U.S. presidential primary races made in California. Use the csv file located here: https://drive.google.com/file/d/1Lgg-PwXQ6TQLDowd6XyBxZw5g1NGWPjB/view?usp=sharing. You should download and save this file in the same folder as this notebook is stored.  This file originally came from the U.S. Federal Election Commission (https://www.fec.gov/).

**DO NOT PUSH THIS FILE TO YOUR GITHUB REPO!**

- Best practice is to not have DATA files in your code repo. As shown below, the default load is outside of the folder this notebook is in. If you change the folder where the file is stored please update the first cell!
- If you do accidentally push the file to your github repo - follow the directions here to fix it: https://docs.google.com/document/d/15Irgb5V5G7pKPWgAerH7FPMpKeQRunbNflaW-hR2hTA/edit?usp=sharing

Documentation for this data can be found here: https://drive.google.com/file/d/11o_SByceenv0NgNMstM-dxC1jL7I9fHL/view?usp=sharing

## Data Questions

You are working for a California state-wide election campaign. Your boss wants you to examine historic 2016 election contribution data to see what zipcodes are more supportive of fundraising for your candidate. 

Your boss asks you to filter out some of the records:
- Only use primary 2016 contribution data (more like how your race is).
- Concentrate on Bernie Sanders as a candidate (most a like your candidate)

The questions your boss wants answered is:
- Which zipcode (5-digit zipcode) had the highest count of contributions and the most dollar amount?
- What day(s) of the month do most people donate?

## Setup

Run the cell below as it will load the data into a pandas dataframe named `contrib`. Note that a custom date parser is defined to speed up loading. If Python were to guess the date format, it would take even longer to load.

In [151]:
import pandas as pd
import numpy as np
from datetime import datetime

# These commands below set some options for pandas and to have matplotlib show the charts in the notebook
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# Define a date parser to pass to read_csv
d = lambda x: datetime.strptime(x, '%d-%b-%y')

# Load the data
# We have this defaulted to the folder OUTSIDE of your repo - please change it as needed
contrib = pd.read_csv('P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)

# Note - for now, it is okay to ignore the warning about mixed types. 

/tmp/ipykernel_798/2050583822.py:14: DtypeWarning: Columns (6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  contrib = pd.read_csv('P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)


***
## 1. Initial Data Checks (50 points)

First we will take a preliminary look at the data to check that it was loaded correctly and contains the info we need.

The questions to answer at the end of this section:
- Do we have the correct # of columns and rows. 
- Do the records contain data for the questions we want to answer 
- What columns are important? 
- What columns can be dropped?
- What are the data problems?

**1a.** Print the *shape* of the data. Does this match the expectation? (2 points)

In [152]:
# 1a YOUR CODE HERE
contrib.shape

(1125659, 18)

- **1a answer here:** Yes

**1b.** Print a list of column names. Are all the columns included that are in the documentation? (2 points)

In [153]:
# 1b YOUR CODE HERE
contrib.columns

Index(['cmte_id', 'cand_id', 'cand_nm', 'contbr_nm', 'contbr_city',
       'contbr_st', 'contbr_zip', 'contbr_employer', 'contbr_occupation',
       'contb_receipt_amt', 'contb_receipt_dt', 'receipt_desc', 'memo_cd',
       'memo_text', 'form_tp', 'file_num', 'tran_id', 'election_tp'],
      dtype='object')

- **1b answer here:** Yes

**1c** Print out the first five rows of the dataset. How do the columns `cand_id`, `cand_nm` and `contbr_st` look? (3 points)

In [154]:
# 1c YOUR CODE HERE
contrib.head()

,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
0,C00575795,P00003392,"Clinton, Hillary Rodham","AULL, ANNE",LARKSPUR,CA,"949,391,913.00",NaN,RETIRED,50.00,2016-04-26,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4768722,P2016
1,C00575795,P00003392,"Clinton, Hillary Rodham","CARROLL, MARYJEAN",CAMBRIA,CA,"934,284,638.00",NaN,RETIRED,200.00,2016-04-20,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4747242,P2016
2,C00575795,P00003392,"Clinton, Hillary Rodham","GANDARA, DESIREE",FONTANA,CA,"923,371,507.00",NaN,RETIRED,5.00,2016-04-02,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4666603,P2016
3,C00577130,P60007168,"Sanders, Bernard","LEE, ALAN",CAMARILLO,CA,"930,111,214.00",AT&T GOVERNMENT SOLUTIONS,SOFTWARE ENGINEER,40.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKWA097,P2016
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,"902,784,310.00",VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016


- **1c answer here:** cand_id and cand_nm seem to be linked, contbr_st seems to be all the same value (CA)

**1d.** Print out the values for the column `election_tp`. In your own words, based on the documentation, what information does the `election_tp` variable contain? Do the values in the column match the documentation? (3 points)

In [155]:
# 1d YOUR CODE HERE
contrib['election_tp']

0          P2016
1          P2016
2          P2016
3          P2016
4          P2016
           ...  
1125654    P2016
1125655    P2016
1125656    P2016
1125657    P2016
1125658    P2016
Name: election_tp, Length: 1125659, dtype: object

- **1d answer here:** Seems like it is the type of election (general, primary, etc). Looks like it does match the documentation

**1e.** Print out the datatypes for all of the columns. What are the datatypes for the `contbr_zip`, `contb_receipt_amt`, `contb_receipt_dt`? (5 points)

In [156]:
# 1e YOUR CODE HERE
contrib.dtypes

cmte_id                      object
cand_id                      object
cand_nm                      object
contbr_nm                    object
contbr_city                  object
contbr_st                    object
contbr_zip                   object
contbr_employer              object
contbr_occupation            object
contb_receipt_amt           float64
contb_receipt_dt     datetime64[ns]
receipt_desc                 object
memo_cd                      object
memo_text                    object
form_tp                      object
file_num                      int64
tran_id                      object
election_tp                  object
dtype: object

- **1e answer here:** contbr_zip: object, contb_receipt_amt: float64, contb_receipt_dt: datetime64[ns]

**1f.** What columns have the most non-nulls?  Would you recommend to drop any columns based on the number of nulls? (5 points)

In [157]:
# 1f YOUR CODE HERE
contrib.count()

cmte_id              1125659
cand_id              1125659
cand_nm              1125659
contbr_nm            1125659
contbr_city          1125633
contbr_st            1125659
contbr_zip           1125564
contbr_employer       967757
contbr_occupation    1115260
contb_receipt_amt    1125659
contb_receipt_dt     1125659
receipt_desc           15045
memo_cd               144268
memo_text             501148
form_tp              1125659
file_num             1125659
tran_id              1125659
election_tp          1124234
dtype: int64

- **1f answer here:** 10 columns have every cell filled with some value. The receipt_desc column only has 15045 non-null values, meaning that ~1.3% of cells are filled. I would recommend to drop this column. Other columns with a low amount of null values are memo_text, memo_cd, and contbr_employer. 

**1g.** A column we know that we want to use is the cand_nm column.  From the documentation each candidate is a unique candidate id also. Check data quality of `cand_id` column to see if it matches `cand_nm` column. Specifically check to ensure our targetted candidate 'Bernard Sanders' always has the same cand_id throughout. Any issues with `cand_nm` matching `cand_id`? (5 points)

In [158]:
# 1g YOUR CODE HERE
id_name_match = contrib['cand_id'] == contrib['cand_nm']
mismatched_rows = contrib[id_name_match]

if mismatched_rows.empty:
    print("All IDs match the corresponding names.")
else:
    print("Mismatched rows:")
    print(mismatched_rows)

All IDs match the corresponding names.


- **1g answer here:** Looks like all IDs match their names

**1h.** Another area to check is to make sure all of the records are from California. Check the `contbr_st` column - are there any records outside of California based on `contbr_st`? (5 points)

In [159]:
# 1h YOUR CODE HERE
state = 'contbr_st' 

if contrib['contbr_st'].nunique() == 1:
    print(f"The column contbr_st has all the same values (CA).")
else:
    print(f"The column contbr_st has different values.")

The column contbr_st has all the same values (CA).


- **1h answer here:** All rows have the same value

**1i.** The next column to check for the analysis is the `tran_id` column. This column could be the primary key so look for duplicates. How many duplicate entries are there? Any pattern for why are there duplicate entries? (5 points)

In [160]:
# 1i YOUR CODE HERE
unique_values_count = contrib['tran_id'].nunique()
total_values_count = len(contrib)

if unique_values_count == total_values_count:
    print(f"The column tran_id has all unique values.")
else:
    duplicate_values_count = total_values_count - unique_values_count
    print(f"The column tran_id has {duplicate_values_count} duplicate values.")

The column tran_id has 3454 duplicate values.


- **1i answer here:** The loop shows that there are 3454 unique values

**1j.** Another column to check is the `contb_receipt_amt` that shows the donation amounts. How many negative donations are included? What do negative donations mean? Please show at least pull a few rows to look at the records with negative donations. Do these records match with the expectation of why a negative donation would happen? (5 points)

In [161]:
# 1j YOUR CODE HERE
negatives = 'contb_receipt_amt'  

negative_values_count = (contrib[negatives] < 0).sum()
negatives_df = contrib[contrib['contb_receipt_amt'] < 0]
print(f"The column '{negatives}' has {negative_values_count} negative values.")
negatives_df.head()

The column 'contb_receipt_amt' has 11896 negative values.


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
19,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-25.00,2016-04-29,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826482B,P2016
23,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-150.00,2016-04-29,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826483B,P2016
81,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-60.00,2016-04-14,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1827494,P2016
190,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","NOWELL, DIANA L.",RANCHO SANTA MARGARITA,CA,"926,884,928.00",CAPISTRAND UNIFIED SCHOOL DISTRICT,LIBRARIAN TECHNICIAN,-100.00,2016-04-11,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1639830B,P2016
213,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,-25.00,2016-04-30,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826888B,P2016


- **1j answer here:** There are 11,896 donations. The reason as to why the donation is negative can be found in the memo text or receipt description: It says 'REDESIGNATION TO GENERAL', which may mean some kind of reallocation or repayment for overages. 

**1k.** One more column to look at is the date of donation column. Are there any dates outside of the primary period (defined as 1 Jan 2014 to 7 June 2016)? Are the dates well-formatted for our analysis? (5 points)

In [162]:
# 1k YOUR CODE HERE
dates = 'contb_receipt_dt'
contrib[dates] = pd.to_datetime(contrib[dates], errors='coerce')

start_date = datetime(2014, 1, 1)
end_date = datetime(2016, 6, 7)

outside_date_range_df = contrib[(contrib[dates] < start_date) | (contrib[dates] > end_date)]
count_outside_date_range = len(outside_date_range_df)

print(f"The number of rows outside the date range is: {count_outside_date_range}")
outside_date_range_df.head(10)

The number of rows outside the date range is: 451372


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
9932,C00458844,P60006723,"Rubio, Marco","WHEELER, MARY MS.",ATHERTON,CA,"940,273,415.00",SELF-EMPLOYED,INTERIOR DESIGNER,-20.00,2013-11-05,NaN,X,TRANSFER FROM RUBIO VICTORY,SA18,1029436,SA18.631526.2.0615,P2016
9994,C00458844,P60006723,"Rubio, Marco","WHEELER, MARY MS.",ATHERTON,CA,"940,273,415.00",SELF-EMPLOYED,INTERIOR DESIGNER,20.00,2013-11-05,NaN,X,TRANSFER FROM RUBIO VICTORY,SA18,1029436,SA18.631526.3.0615,G2016
14673,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","RANDALL, DICK J",CUPERTINO,CA,"95,014.00",NaN,NaN,"-5,400.00",2016-06-30,Refund,NaN,NaN,SB28A,1096256,SB28A.90932,NaN
14682,C00605568,P20002671,"Johnson, Gary","BOAL, ROB",OAK PARK,CA,"91,377.00",VENDAVO,ENGINEER,30.00,2016-07-20,NaN,NaN,NaN,SA17A,1096305,SA17A.50793,G2016
14697,C00605568,P20002671,"Johnson, Gary","LEE, JASCHA",SANTA CRUZ,CA,"95,065.00",YAHOO,S/W ENG,88.45,2016-07-14,NaN,NaN,NaN,SA17A,1096305,SA17A.51414,G2016
14698,C00605568,P20002671,"Johnson, Gary","LEE, JASCHA",SANTA CRUZ,CA,"95,065.00",YAHOO,S/W ENG,100.00,2016-07-24,NaN,NaN,NaN,SA17A,1096305,SA17A.51415,G2016
14700,C00575795,P00003392,"Clinton, Hillary Rodham","BAKER, JUDY",VACAVILLE,CA,"956,873,433.00",NaN,RETIRED,5.00,2016-07-25,NaN,X,* HILLARY VICTORY FUND,SA18,1109498,C8807328,P2016
14738,C00575795,P00003392,"Clinton, Hillary Rodham","JOHNSTON, NAN",PACIFIC GROVE,CA,"939,504,535.00",RETIRED,RN,52.12,2016-07-04,NaN,X,* HILLARY VICTORY FUND,SA18,1109498,C8701938,P2016
14739,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","VAUGHN, J MICHAEL",TRABUCO CANYON,CA,"92,679.00",NaN,NaN,"-1,000.00",2016-06-10,Refund,NaN,NaN,SB28A,1096256,SB28A.90913,NaN
14745,C00605568,P20002671,"Johnson, Gary","VANDERVEEN, DAVID",LAGUNA BEACH,CA,"92,651.00",TWICE REQUESTED NOT RECEIVED,TWICE REQUESTED NOT RECEIVED,500.00,2016-07-19,NaN,NaN,NaN,SA17A,1096305,SA17A.52002,G2016


- **1k answer here:**

**1l.** Finally, answer the initial questions in the cells below (5 points)

**1l.1** Do we have the correct # of columns and rows.

- **1l.1 answer here:** Yes

**1l.2** Do the records contain data for the questions we want to answer?

- **1l.2 answer here:** Yes

**1l.3** What columns are important?

- **1l.3 answer here:** cand_nm, contbr_nm, contbr_city, contbr_occupation, contbr_receipt_amt, memo_text

**1l.4** What columns can be dropped?

- **1l.4 answer here:** contbr_st, receipt_desc, possibly file_num? 

**1l.5** What are the data problems?

- **1l.5 answer here:** Many null values in a few columns, must be cleaned. Also many duplicate values in the transaction column, and if it will be used as a primary key, this will not suffice. 

**1l.6** List any assumptions so far:

- **1l.6 answer here:** That there are no inaccurate data points, nothing is misspelled or miscounted 

***
## 2. Data filtering and data quality fixes (30 points)

Now that we have a basic understanding of the data, let's filter out the records we don't need and fix the data.

**2a.** From the dataset filter out (remove) any election_tp not in the primary election. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [169]:
# 2a YOUR CODE HERE
primary = 'P2016' 
filtered_df = contrib[~contrib['election_tp'].astype(str).str.contains(primary, case=False, na=False)]

print(filtered_df.shape)
filtered_df.head()

(315178, 18)


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
18,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,25.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826482,G2016
82,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,60.00,2016-04-14,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1827629,G2016
89,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,150.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826483,G2016
212,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,25.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826888,G2016
216,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,50.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826889,G2016


**2b.** From the dataset filter out (remove) any candidate that is not Bernie Sanders. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [170]:
# 2b YOUR CODE HERE
bernie = 'Sanders, Bernard'
filtered_df = filtered_df[~filtered_df['cand_nm'].astype(str).str.contains(bernie, case=False, na=False)]

print(filtered_df.shape)
filtered_df.head()

(315177, 18)


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
18,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,25.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826482,G2016
82,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,60.00,2016-04-14,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1827629,G2016
89,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,150.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826483,G2016
212,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,25.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826888,G2016
216,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,50.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826889,G2016


**2c.** The `contbr_zip` column is not formatted well for our analysis. Make a new zipcode column that is the five-digit zipcodes. Filter out any records outside of California based on the zipcode. Print/show the shape of the dataframe after the filtering is complete. (10 points).

- You will have to research what the valid 5-digit zipcodes for California are!

In [171]:
# 2c YOUR CODE HERE
filtered_df['contbr_zip'] = filtered_df['contbr_zip'].astype(str)
filtered_df['five_zipcode'] = filtered_df['contbr_zip'].str[:5]
filtered_df = filtered_df[filtered_df['five_zipcode'].str.startswith('9')]

print("Shape of the DataFrame after filtering for California zip codes:", filtered_df.shape)
filtered_df.head()

Shape of the DataFrame after filtering for California zip codes: (315075, 19)


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp,five_zipcode
18,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,959289507.0,SELF EMPLOYED,RANCHER,25.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826482,G2016,95928
82,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,959289507.0,SELF EMPLOYED,RANCHER,60.00,2016-04-14,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1827629,G2016,95928
89,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,959289507.0,SELF EMPLOYED,RANCHER,150.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826483,G2016,95928
212,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,921096720.0,SELF EMPLOYED,REAL ESTATE,25.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826888,G2016,92109
216,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,921096720.0,SELF EMPLOYED,REAL ESTATE,50.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826889,G2016,92109


**2d.** The receipt amount column has negative donations. After talking with your team, a decision was made that the best course of action is to remove these negative values so that the donation count and amount is more accurate. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [174]:
# 2d YOUR CODE HERE
non_negative_amts = 'contb_receipt_amt'

# Filter out rows with negative values
filtered_df = filtered_df[filtered_df['contb_receipt_amt'] >= 0]

print(filtered_df.shape)
filtered_df.head()

(311722, 19)


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp,five_zipcode
18,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,959289507.0,SELF EMPLOYED,RANCHER,25.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826482,G2016,95928
82,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,959289507.0,SELF EMPLOYED,RANCHER,60.00,2016-04-14,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1827629,G2016,95928
89,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,959289507.0,SELF EMPLOYED,RANCHER,150.00,2016-04-29,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826483,G2016,95928
212,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,921096720.0,SELF EMPLOYED,REAL ESTATE,25.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826888,G2016,92109
216,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,921096720.0,SELF EMPLOYED,REAL ESTATE,50.00,2016-04-30,REDESIGNATION FROM PRIMARY,X,REDESIGNATION FROM PRIMARY,SA17A,1077664,SA17A.1826889,G2016,92109


**2e.** From the dataset drop any columns that won't be used in the analysis. Print/show the shape of the dataframe after the dropping is complete. What columns did you drop and why? (5 points)

In [175]:
# 2e YOUR CODE HERE

filtered_df.drop(columns='contbr_st', inplace=True)
filtered_df.drop(columns='receipt_desc', inplace=True)
filtered_df.drop(columns='file_num', inplace=True)


- **2e answer here:** Dropped the state column because it is constant across all rows, drop the resceipt_desc because only 1% of rows have a non-null value, and dropped the file number column because there is no explanation in the documentation as to what it represents. 

**2f.** List any assumptions that you made up to this point:

- **2f answer here:** All data is accurate, correctly spelled, counted. Data has been collected correctly and methodically. 

***
## 3. Answering the questions (20 points)

Now that the data is cleaned and filterd - let's answer the two questions from your boss!

**3a.** Which zipcode had the highest count of contributions and the most dollar amount? (10 points)

In [181]:
# 3a YOUR CODE HERE
result_df = filtered_df.groupby('five_zipcode')['contb_receipt_amt'].sum().reset_index()
max_row = result_df.loc[result_df['contb_receipt_amt'].idxmax()]
highest_contributions_count = filtered_df[filtered_df['five_zipcode'] == max_row['five_zipcode']]['contb_receipt_amt'].count()

print(result_df)
print(max_row)
print(highest_contributions_count)

     five_zipcode  contb_receipt_amt
0             9.0              50.00
1           90001           2,986.00
2           90002             612.00
3           90003             592.00
4           90004         247,736.61
...           ...                ...
1935        97209             500.00
1936        97305             500.00
1937        98166             125.00
1938        98464           1,500.00
1939        99999          27,327.20

[1940 rows x 2 columns]
five_zipcode             94114
contb_receipt_amt   702,170.21
Name: 1153, dtype: object
3386


- **3a answer here:** The zipcode with the highest total sum was 94114 at a total of $702,170.21 and had 3386 contributions. 

**3b.** What day(s) of the month do most people donate? (10 points)

In [182]:
# 3b YOUR CODE HERE
filtered_df['contb_receipt_dt'] = pd.to_datetime(filtered_df['contb_receipt_dt'])
filtered_df['day_of_month'] = filtered_df['contb_receipt_dt'].dt.day
daily_contributions = filtered_df.groupby('day_of_month')['contb_receipt_amt'].sum().reset_index()
max_contributions_day = daily_contributions.loc[daily_contributions['contb_receipt_amt'].idxmax()]

print(max_contributions_day)

day_of_month               30.00
contb_receipt_amt   2,635,953.26
Name: 29, dtype: float64


- **3b answer here:** the 30th of the month 

## If you have feedback for this homework, please submit it using the link below:

http://goo.gl/forms/74yCiQTf6k